# 本地化大模型部署与验证

本 Notebook 包含以下步骤：
1. **环境检查**: 确认 GPU 和 CUDA 环境是否满足要求。
2. **模型下载**: 从 ModelScope 下载 Qwen2.5-7B-Instruct-GPTQ-Int4 量化模型。
3. **模型加载**: 加载 Tokenizer 和模型到显存。
4. **推理测试**: 定义对话函数并进行测试。

**关键问题解决记录**：
- **问题**：初始部署推理延迟极高（>40秒）

- **参考文章**：[解决 AutoGPTQ 推理慢的问题](https://jishuzhan.net/article/1868560413910634498)
- **根本原因**：AutoGPTQ 0.7.1 需要 PyTorch 2.2.1+，而环境中是 PyTorch 2.1.2，导致 CUDA 扩展未正确加载，回退到 CPU 执行。
- **解决方案**：降级到 AutoGPTQ 0.6.0（适配 PyTorch 2.1.2）。
- **结果**：性能提升 25 倍（推理时间从 100s 降至 4s）。

## 1. conda环境配置与环境检查

In [6]:
! conda env create -f environment.yml

Channels:
 - pytorch
 - nvidia
 - conda-forge
 - defaults
Platform: win-64
Solving environment: done
Installing pip dependencies: \ Ran pip subprocess with arguments:
['D:\\anaconda\\envs\\llm_deploy\\python.exe', '-m', 'pip', 'install', '-U', '-r', 'd:\\llm_deploy\\LLM\\LLM_DEPLOY\\condaenv.3qahz6wx.requirements.txt', '--exists-action=b']
Pip subprocess output:
Looking in indexes: https://download.pytorch.org/whl/cu121














































































































done
#
# To activate this environment, use
#
#     $ conda activate llm_deploy
#
# To deactivate an active environment, use
#
#     $ conda deactivate





==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda




In [ ]:
import torch
import sys

print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")

if torch.cuda.is_available():
    print(f"CUDA is available!")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    # Check VRAM
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"Total VRAM: {total_memory:.2f} GB")
else:
    print("WARNING: CUDA is NOT available. Inference will be extremely slow on CPU.")

Python Version: 3.10.19 | packaged by conda-forge | (main, Oct 22 2025, 22:23:22) [MSC v.1944 64 bit (AMD64)]
PyTorch Version: 2.1.2+cu121
CUDA is available!
GPU Device: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA Version: 12.1
Total VRAM: 8.00 GB


## 2. 模型下载

使用 **Qwen2.5-7B-Instruct-GPTQ-Int4** 量化模型：
- 模型大小：约 4.3GB（Int4 量化）

- 显存占用：约 5-6GB- 结果：性能提升 12 倍

- 推理速度：约 3-4 秒/次（修复后）- 解决：AutoGPTQ 0.7.1 → 0.6.0，匹配 PyTorch 2.1.2

- 适配硬件：RTX 4060 8GB- 原因：AutoGPTQ CUDA 扩展未加载（版本不匹配）

- 问题：初始部署推理延迟 >40 秒
**性能优化记录**：

In [ ]:
from modelscope import snapshot_download
import os

# 确保下载目录存在
os.makedirs('./models', exist_ok=True)

# 下载 Qwen2.5-7B GPTQ Int4 模型
model_id = 'Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4'

print(f"开始下载 {model_id} 模型...")
try:
    model_dir = snapshot_download(
        model_id, 
        cache_dir='./models',
        revision='master'
    )
    print(f"模型下载成功！存储路径: {model_dir}")
except Exception as e:
    print(f"模型下载失败: {e}")

开始下载 Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4 模型...


2025-12-14 20:35:50,628 - modelscope - INFO - Creating symbolic link [./models\Qwen\Qwen2.5-7B-Instruct-GPTQ-Int4].
2025-12-14 20:35:50,638 - modelscope - WARNING - Failed to create symbolic link ./models\Qwen\Qwen2.5-7B-Instruct-GPTQ-Int4 for d:\llm_deploy\models\Qwen\Qwen2___5-7B-Instruct-GPTQ-Int4.
2025-12-14 20:35:50,638 - modelscope - WARNING - Failed to create symbolic link ./models\Qwen\Qwen2.5-7B-Instruct-GPTQ-Int4 for d:\llm_deploy\models\Qwen\Qwen2___5-7B-Instruct-GPTQ-Int4.


模型下载成功！存储路径: ./models\Qwen\Qwen2___5-7B-Instruct-GPTQ-Int4


## 3. 模型加载
加载下载的模型。

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from modelscope import snapshot_download
import torch

model_id = 'Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4'

# 获取模型路径
model_dir = snapshot_download(model_id, cache_dir='./models')

print(f"正在加载 Tokenizer 和 模型 ({model_id})...")
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

# 加载 GPTQ 量化模型 (AutoGPTQ 0.6.0 会自动处理)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
)
print("模型加载完成！")

2025-12-14 20:35:54,300 - modelscope - INFO - Creating symbolic link [./models\Qwen\Qwen2.5-7B-Instruct-GPTQ-Int4].
2025-12-14 20:35:54,313 - modelscope - WARNING - Failed to create symbolic link ./models\Qwen\Qwen2.5-7B-Instruct-GPTQ-Int4 for d:\llm_deploy\models\Qwen\Qwen2___5-7B-Instruct-GPTQ-Int4.
2025-12-14 20:35:54,313 - modelscope - WARNING - Failed to create symbolic link ./models\Qwen\Qwen2.5-7B-Instruct-GPTQ-Int4 for d:\llm_deploy\models\Qwen\Qwen2___5-7B-Instruct-GPTQ-Int4.


正在加载 Tokenizer 和 模型 (Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4)...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

模型加载完成！


## 4. 定义推理函数
定义一个 `chat` 函数，封装 Prompt 构建和生成过程。

In [7]:
def chat(query):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": query}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.8,
        top_k=20,
        do_sample=True
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

## 5. 测试对话

In [ ]:
test_query = "你好，请介绍一下你自己。"
print(f"\nUser: {test_query}")
# chat 函数已经返回了字符串，直接打印即可，不需要再用 print 包裹
response = chat(test_query)
print(f"Assistant: {response}")


User: 你好，请介绍一下你自己。


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Assistant: 你好！我是一个来自阿里云的语言模型，我叫通义千问。我可以帮助用户生成各种类型的文本，例如文章、故事、诗歌、故事等，并能够回答各种问题。尽管我非常强大，但我也不是完美无缺的，我还需要不断学习和进步。如果你有任何问题或需要帮助，都可以随时向我提问。


In [ ]:
# 使用经典问题来测试下
test_query_2 = "鲁迅和周树人是什么关系？"
print(f"\nUser: {test_query_2}")
response_2 = chat(test_query_2)
print(f"Assistant: {response_2}")


User: 鲁迅和周树人是什么关系？
Assistant: 鲁迅是周树人的笔名。鲁迅是中国近现代著名的文学家、思想家、革命家，原名周树人，浙江绍兴人。1918年5月，首次用“鲁迅”作为自己的笔名，发表中国现代文学史上第一篇白话文小说《狂人日记》。
Assistant: 鲁迅是周树人的笔名。鲁迅是中国近现代著名的文学家、思想家、革命家，原名周树人，浙江绍兴人。1918年5月，首次用“鲁迅”作为自己的笔名，发表中国现代文学史上第一篇白话文小说《狂人日记》。
